In [23]:
import great_expectations as gx
import pandas as pd
context = gx.get_context()

In [25]:
context

{
  "checkpoint_store_name": "checkpoint_store",
  "config_variables_file_path": "uncommitted/config_variables.yml",
  "config_version": 4.0,
  "data_context_id": "7b50778a-db42-43d6-8859-b869390232d3",
  "data_docs_sites": {
    "local_site": {
      "class_name": "SiteBuilder",
      "show_how_to_buttons": true,
      "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": "uncommitted/data_docs/local_site/"
      },
      "site_index_builder": {
        "class_name": "DefaultSiteIndexBuilder"
      }
    }
  },
  "expectations_store_name": "expectations_store",
  "fluent_datasources": {},
  "plugins_directory": "plugins/",
  "stores": {
    "expectations_store": {
      "class_name": "ExpectationsStore",
      "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": "expectations/"
      }
    },
    "validation_results_store": {
      "class_name": "ValidationResultsStore",
      "store_backend": {
 

In [28]:
# Step 1: Load Data
df = pd.read_csv("./data/wind_updated_file1.csv")

In [29]:
context = gx.get_context()

# Step 3: Add a Pandas Data Source
# Note: If 'add_pandas' doesn't exist, make sure to use the correct method to add a datasource.
data_source = context.data_sources.add_pandas(name="ds1")

# Step 4: Add DataFrame as a Data Asset
# The asset is defined using the name "pandas_asset"
data_asset = data_source.add_dataframe_asset(name="da1")

# Step 5: Add Batch Definition
# Creating a batch definition for the whole dataframe
batch_definition = data_asset.add_batch_definition_whole_dataframe("bd1")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/posthog/client.py:310: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/posthog/request.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()


In [30]:
suite_name = "es1"
suite = gx.ExpectationSuite(name=suite_name)

# Add the expectation suite to the context
suite = context.suites.add(suite)


e1 = gx.expectations.ExpectColumnValuesToBeBetween(
    column="plf",  # Ensure this column name is correct and matches your dataset
    min_value=20,
    max_value=35,
    meta={"notes": "Values between 20 and 35 are valid for `plf`."}
)
e2 = gx.expectations.ExpectColumnValuesToBeBetween(
    column="plf",  # Make sure the column name is correct and matches the data
    min_value=20,
    max_value=35,
    meta={"notes": "Values between 20 and 35 are valid for `plf`."}
)
suite.add_expectation(e1)

# Add the expectation to the suite
suite.add_expectation(e2)

ExpectColumnValuesToBeBetween(id=None, meta={'notes': 'Values between 20 and 35 are valid for `plf`.'}, notes=None, result_format=<ResultFormat.BASIC: 'BASIC'>, description=None, catch_exceptions=True, rendered_content=None, windows=None, batch_id=None, column='plf', mostly=1, row_condition=None, condition_parser=None, min_value=20.0, max_value=35.0, strict_min=False, strict_max=False)

In [31]:
definition_name = "vd1"
validation_definition = gx.ValidationDefinition(
    data=batch_definition,  # Use 'batch' here as the actual data for validation
    suite=suite,
    name=definition_name
    
)

# Add the validation definition to the context
validation_definition = context.validation_definitions.add(validation_definition)

In [ ]:
# batch_parameters = {"dataframe": df}
# # Step 9: Run the Validation Definition
# validation_results = validation_definition.run(batch_parameters=batch_parameters)

# # Step 10: Review the Validation Results
# print(validation_results)

In [ ]:
import great_expectations as gx
from great_expectations.checkpoint import (
    MicrosoftTeamsNotificationAction,
    UpdateDataDocsAction,
)


# Create a list of one or more Validation Definitions for the Checkpoint to run
validation_definitions = [
    context.validation_definitions.get("vd1")
]

In [42]:
# Define the Microsoft Teams webhook URL
teams_webhook_url = "https://sustainabilityeconomics736.webhook.office.com/webhookb2/3c4e89fb-5045-4b2f-9623-977b9e118f7d@a8606430-fd02-4b66-b04e-e614589bb5ae/IncomingWebhook/b92a1af86433472689f0ab8a7547d535/861906e0-eea2-4460-b1d0-099f375856b3/V2F6iHE8JUnVF62SMSgT91RQDYNuCZ71HKBFiTiNBZx581"  # Replace with your Teams webhook

# Create a list of Actions for the Checkpoint to perform
action_list = [
    # Microsoft Teams Notification Action
    MicrosoftTeamsNotificationAction(
        name="send_teams_notification",
        teams_webhook=teams_webhook_url,
        notify_on="success",  # Trigger notification on "failure"
    ),
    # Update Data Docs Action
    UpdateDataDocsAction(
        name="update_all_data_docs",
    ),
]

# Create the Checkpoint
checkpoint_name = "teams_notifications_checkpoint2"
checkpoint = gx.Checkpoint(
    name=checkpoint_name,
    validation_definitions=validation_definitions,
    actions=action_list,
    result_format={"result_format": "COMPLETE"},
)

# Save the Checkpoint to the Data Context
context.checkpoints.add(checkpoint)

Checkpoint(name='teams_notifications_checkpoint2', validation_definitions=[ValidationDefinition(name='vd1', data=BatchDefinition(id=UUID('bea83117-6e1c-4ae9-bda8-1d70c456ca4c'), name='bd1', partitioner=None), suite={
  "name": "es1",
  "id": "a6038641-9e67-4db1-abd7-5c0db46567ea",
  "expectations": [
    {
      "type": "expect_column_values_to_be_between",
      "kwargs": {
        "column": "plf",
        "min_value": 20.0,
        "max_value": 35.0
      },
      "meta": {
        "notes": "Values between 20 and 35 are valid for `plf`."
      },
      "id": "fdb6714e-2e92-40d7-a5d6-739ecca1e98b"
    }
  ],
  "meta": {
    "great_expectations_version": "1.2.4"
  },
  "notes": null
}, id='97c002df-7a83-40ad-9b87-1893dce097db')], actions=[MicrosoftTeamsNotificationAction(type='microsoft', name='send_teams_notification', teams_webhook='https://sustainabilityeconomics736.webhook.office.com/webhookb2/3c4e89fb-5045-4b2f-9623-977b9e118f7d@a8606430-fd02-4b66-b04e-e614589bb5ae/IncomingWebhook

In [43]:
# Retrieve the Checkpoint later
checkpoint_name = "teams_notifications_checkpoint2"
checkpoint = context.checkpoints.get(checkpoint_name)

batch_parameters = {"dataframe": df}

# Run the Checkpoint
results = checkpoint.run(batch_parameters=batch_parameters)

# Output the results
print("Checkpoint results:", results)

Calculating Metrics:   0%|          | 0/10 [00:00<?, ?it/s]

Checkpoint results: run_id={"run_name": null, "run_time": "2024-12-04T10:15:17.542838+05:30"} run_results={ValidationResultIdentifier::es1/__none__/20241204T044517.542838Z/ds1-da1: {
  "success": true,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_be_between",
        "kwargs": {
          "batch_id": "ds1-da1",
          "column": "plf",
          "min_value": 20.0,
          "max_value": 35.0
        },
        "meta": {
          "notes": "Values between 20 and 35 are valid for `plf`."
        },
        "id": "fdb6714e-2e92-40d7-a5d6-739ecca1e98b"
      },
      "result": {
        "element_count": 63,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": [],
        "missing_count": 0,
        "missing_percent": 0.0,
        "unexpected_percent_total": 0.0,
        "unexpected_percent_nonmissing": 0.0,
        "partial_unexpected_counts": [],
        "partial_unexpec

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/posthog/client.py:310: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/posthog/request.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()


In [44]:
context.open_data_docs()